# The Impact of the Coronavirus Pandemic on Connecticut's Residential Real Estate Market

I'll be using this file to do all the necessary data wrangling, analysis and plotting with python, and maybe even take a few notes that will go to the final paper.

#### Objectives
- [x] import dataset
- [x] add data about aggregate infection numbers in Connecticut
- [x] add data about income numbers in CT
- [ ] merge house sales, covid cases and income tables to one dataframe
- [ ] describe where we got the data from
- [ ] show table summary
- [ ] answer questions using plots and graphs

#### Questions
The background goes over the various questions we want to answer with this paper. Specifically, we want to investigate whatever demographic data we can find to explain income, along with infection rates, affected the residential real estate market in CT. A few questions we have to start include the following:
1. What do the sale prices in each quartile look like?
2. What is the trend in sales from 2016 to 2018?
3. How does infection affect sales in each quartile?
4. What cities show the largest changes? Why do we think that was?
We might have to define the quartiles.

In [ ]:
# install dependencies
# !pip install --upgrade pip

In [1]:
# import modules
from IPython.display import display

import pandas as pd

# import datasets
data = pd.read_csv('./data/CTRRE_2011-2021.csv')
infection_data = pd.read_csv('./data/COVID19_Tests_Cases_Deaths_by_Town-ARCHIVE.csv')
income_data = pd.read_csv('./data/median-household-income-town-2020.csv')

# preview primary dataset
display(data.head())

,list_year,town,population,residential_type,month,year,in_pandemic,assessed_value,sale_amount,price_index,norm_assessed_value,norm_sale_amount,norm_sales_ratio,latitude,longitude
0,2020,Ashford,4193,Single Family,10,2020,1,253000,430000.0,254.076,99576.50,169240.70,0.588372,41.8731,-72.1216
1,2020,Avon,18821,Condo,3,2021,1,130400,179900.0,258.935,50360.13,69476.90,0.724847,41.8096,-72.8305
2,2020,Avon,18821,Single Family,4,2021,1,619290,890000.0,261.237,237060.60,340686.81,0.695831,41.8096,-72.8305
3,2020,Avon,18821,Single Family,7,2021,1,862330,1447500.0,267.789,322018.45,540537.51,0.595737,41.8096,-72.8305
4,2020,Avon,18821,Single Family,12,2020,1,847520,1250000.0,254.081,333562.92,491969.10,0.678016,41.8096,-72.8305


We want to add 2016 to 2020 median household income from the cities in Connecticut, as well as relevant data about total covid infection numbers from the cities.

In [2]:
# preview infection dataset
display(infection_data.tail())

,Last update date,Town number,Town,Total cases,Confirmed cases,Probable cases,Case rate,Total deaths,Confirmed deaths,Probable deaths,People tested,Rate tested per 100k,Number of tests,Number of positives,Number of negatives,Number of indeterminates
102240,06/24/2022,165,Windsor Locks,2702,2507.0,195.0,21021.0,44,40.0,4.0,10507.0,81741.0,44778.0,3347.0,41376.0,55.0
102241,06/24/2022,166,Wolcott,4089,3557.0,532.0,24652.0,64,54.0,10.0,14324.0,86357.0,71416.0,5163.0,66148.0,105.0
102242,06/24/2022,167,Woodbridge,1699,1519.0,180.0,19417.0,48,42.0,6.0,8668.0,99063.0,50559.0,2115.0,48373.0,71.0
102243,06/24/2022,168,Woodbury,1646,1367.0,279.0,17323.0,12,10.0,2.0,7918.0,83330.0,34243.0,2023.0,32192.0,28.0
102244,06/24/2022,169,Woodstock,1440,1391.0,49.0,18325.0,7,7.0,0.0,6740.0,85772.0,23654.0,1680.0,21945.0,29.0


In [3]:
# preview dataset with median household income
display(income_data.head())

,Town,FIPS,Year,Race/Ethnicity,Measure Type,Variable,Value
0,Andover,901301080,2005-2009,All,Number,Median Household Income,84757.00
1,Andover,901301080,2005-2009,All,Number,Margins of Error,9003.00
2,Andover,901301080,2005-2009,All,Ratio to State Median,Median Household Income,1.25
3,Andover,901301080,2005-2009,All,Ratio to State Median,Margins of Error,0.13
4,Andover,901301080,2005-2009,American Indian and Alaska Native Alone,Number,Median Household Income,-9999.00


The data about covid infection rates looks clean and easy enough to merge with the primary dataset. The data about median income households is a little less straightforward, and from the outset, it looks like I definitely need to do some row and column manipulation to get the desired values.

I'll start with wrangling the infection rates dataset and merging it with the primary dataset.

In [145]:
# rename columns for convenience
infection_data = infection_data.rename(columns={
    'Last update date': 'update_date',
    'Town': 'town',
    'Total cases ': 'total_cases'
})

inf_df1 = infection_data.loc[:, ['update_date', 'town', 'total_cases']]
display(inf_df1.head())

,update_date,town,total_cases
0,01/17/2021,Andover,118
1,01/17/2021,Ansonia,1236
2,01/17/2021,Ashford,158
3,01/17/2021,Avon,614
4,01/17/2021,Barkhamsted,115


In [146]:
# convert 'update_date' column to date datatype
# offset every date to the last date of the month
# then filter to get the highest number of cases for each month
inf_df1['update_date'] = pd.to_datetime(inf_df1['update_date'])
inf_df1['month'] = inf_df1['update_date'] + pd.offsets.MonthEnd(0)
inf_df1 = inf_df1.groupby([pd.Grouper(key='month', freq='M'), 'town'])['total_cases'].idxmax().reset_index()
inf_df1 = inf_df1.sort_values(by="month")

display(inf_df1.head())

,month,town,total_cases
0,2020-03-31,Andover,11401
108,2020-03-31,Plainfield,10758
109,2020-03-31,Plainville,36848
110,2020-03-31,Plymouth,36849
111,2020-03-31,Pomfret,36850


In [31]:
# find how many unique values exist in 'Variable' column
# since it has all the income data
#income_data['Variable'].unique()

# we want to pivot the income dataset so that we can isolate the
# 'Median Household Income' value and make it it's own column

# in new 'idx' col, rename every other row to be as the one above it
inc_tmp1 = income_data.set_index(pd.Index([i//2 for i in range(len(income_data))]))
inc_tmp1 = inc_tmp1.rename_axis('idx').reset_index()

# pivot to isolate values in 'Variable' column
inc_tmp1 = inc_tmp1.pivot(index='idx', columns='Variable', values='Value')

# repeat each row but fix indices
inc_tmp1 = inc_tmp1.loc[inc_tmp1.index.repeat(2)].reset_index(drop=True)

# merge tables laterally, then drop every other row
inc_tmp2 = pd.concat([income_data, inc_tmp1], axis=1)
inc_tmp2 = inc_tmp2.iloc[::2]

income_data2 = inc_tmp2.rename(columns={
    'Town': 'town',
    'Year': 'year',
    'Race/Ethnicity': 'demographics',
    'Median Household Income': 'med_hsehld_income'
})
inc_df1 = income_data2.loc[:, ['town', 'year', 'demographics', 'med_hsehld_income']].reset_index()
inc_df1 = inc_df1.drop('index', axis=1)
inc_df1.head()

,town,year,demographics,med_hsehld_income
0,Andover,2005-2009,All,84757.00
1,Andover,2005-2009,All,1.25
2,Andover,2005-2009,American Indian and Alaska Native Alone,-9999.00
3,Andover,2005-2009,American Indian and Alaska Native Alone,-9999.00
4,Andover,2005-2009,Asian Alone,250001.00


I decided to create a sequence of doubly-repeating row numbers to get a single column from the 2 unique values in 'Variable', instead of two rows with `NaN` in both of them where the other's value was supposed to be.

By this point, we have the relevant columns from the income and infections dataset that we could merge with the primary dataset. Unfortunately, the primary dataset with residential housing sales does not have a date variable that we definitely need to merge with the other two.

My solution at the moment is to instead get the necessary rows from the housing dataset and concatenate them with the infections and income datasets appropriately.

Let's get a quick summary of both the infections and income table so we know how to proceed with matching with the sales dataset. You might have already noticed that there are quite a few unreasonable numbers in the income table. We hope to not lose too much data from this table.

In [10]:
# summarize the covid cases table
cov_summary = inf_df1.describe()
display(cov_summary)

,month,total_cases
count,4732,4732.000000
mean,2021-05-15 23:59:59.999999744,51965.761834
min,2020-03-31 00:00:00,181.000000
25%,2020-10-23 06:00:00,23045.500000
50%,2021-05-15 12:00:00,54369.500000
75%,2021-12-07 18:00:00,78397.250000
max,2022-06-30 00:00:00,102243.000000
std,NaN,31197.901314


In [11]:
# summarize the median household income table
inc_summary = inc_df1.describe()
display(inc_summary)

,med_hsehld_income
count,40800.000000
mean,21562.241111
std,47818.093792
min,-9999.000000
25%,-9999.000000
50%,1.000000
75%,50114.000000
max,250001.000000


In [32]:
# create date column to organize dataframe into dates that the sales were made
df1_tmp1 = data
df1_tmp1['month_sold'] = pd.to_datetime(df1_tmp1['year'].astype(str) + df1_tmp1['month'].astype(str), format='%Y%m')
display(df1_tmp1.head())

,list_year,town,population,residential_type,month,year,in_pandemic,assessed_value,sale_amount,price_index,norm_assessed_value,norm_sale_amount,norm_sales_ratio,latitude,longitude,month_sold
0,2020,Ashford,4193,Single Family,10,2020,1,253000,430000.0,254.076,99576.50,169240.70,0.588372,41.8731,-72.1216,2020-10-01
1,2020,Avon,18821,Condo,3,2021,1,130400,179900.0,258.935,50360.13,69476.90,0.724847,41.8096,-72.8305,2021-03-01
2,2020,Avon,18821,Single Family,4,2021,1,619290,890000.0,261.237,237060.60,340686.81,0.695831,41.8096,-72.8305,2021-04-01
3,2020,Avon,18821,Single Family,7,2021,1,862330,1447500.0,267.789,322018.45,540537.51,0.595737,41.8096,-72.8305,2021-07-01
4,2020,Avon,18821,Single Family,12,2020,1,847520,1250000.0,254.081,333562.92,491969.10,0.678016,41.8096,-72.8305,2020-12-01


In [147]:
# sort rows chronologically
df1_tmp1 = df1_tmp1.sort_values(by='month_sold')

# indicate the end of the month for consistency with covid dataset
df1_tmp1['month_sold'] = df1_tmp1['month_sold'] + pd.offsets.MonthEnd(0)

# select relevant columns
df1 = df1_tmp1.loc[:, [
    'list_year', 'town', 'population', 'residential_type',
    'assessed_value', 'sale_amount', 'month_sold'
]]
df1.head()

,list_year,town,population,residential_type,assessed_value,sale_amount,month_sold
175807,2017,Washington,3619,Single Family,2260610,4250000.0,2010-05-31
157600,2017,Bloomfield,21399,Single Family,126000,140000.0,2010-07-31
161780,2017,Bloomfield,21399,Single Family,125580,215000.0,2010-08-31
425987,2010,Southbury,19866,Single Family,256890,300000.0,2010-10-31
411874,2010,Windsor,29424,Single Family,131530,190000.0,2010-10-31


In [148]:
# filter income table to remain with values from the period '2016-2020'
# filter demographics to remain with median income values from all races
inc_df2 = inc_df1.query("year == '2016-2020' & demographics == 'All'")

# drop demographics column and remove every next row
inc_df2 = inc_df2.drop('demographics', axis=1).loc[::2]

display(inc_df2.head())

,town,year,med_hsehld_income
220,Andover,2016-2020,99449.0
460,Ansonia,2016-2020,53709.0
700,Ashford,2016-2020,84909.0
940,Avon,2016-2020,123077.0
1180,Barkhamsted,2016-2020,103500.0


It is reasonable to proceed with only income data for the period of 2016-2020.

It also appears that I need to rename the `month_sold` column to `month` so I can merge the covid and sales datasets.

In [164]:
# rename `months_sold` column to make it easy to merge the dataframes
df2_tmp1 = df1.rename(columns={ 'month_sold': 'month' })

# filter rows to remain with dates after 2016 (dataset goes up to 2020)
df2_tmp2 = df2_tmp1[df2_tmp1['month'] >= '2016-01-01']

# merge the covid cases and sales tables
df2_tmp3 = pd.merge(df2_tmp2, inf_df1, on=['month', 'town'], how='outer')

# we assume that there were 0 covid cases before March 2020
df2_tmp3.loc[df2_tmp3['month'] < '2020-03-01', 'total_cases'] = 0

# concat income table variable `med_hsehld_income` with combined dataset
# use common variable `town`
df2_tmp4 = pd.merge(df2_tmp3, inc_df2[['town', 'med_hsehld_income']], on='town')

display(df2_tmp4.head())

,list_year,town,population,residential_type,assessed_value,sale_amount,month,total_cases,med_hsehld_income
0,2015.0,Bloomfield,21399.0,Single Family,167300.0,219000.0,2016-01-31,0.0,79134.0
1,2015.0,Bloomfield,21399.0,Single Family,119140.0,188000.0,2016-01-31,0.0,79134.0
2,2015.0,Bloomfield,21399.0,Single Family,61600.0,98000.0,2016-01-31,0.0,79134.0
3,2015.0,Bloomfield,21399.0,Condo,28210.0,34000.0,2016-01-31,0.0,79134.0
4,2015.0,Bloomfield,21399.0,Single Family,91980.0,121248.0,2016-01-31,0.0,79134.0


### Wrangling the resulting dataset

We notice a few things when we preview the dataset. First is that every other row carries information that doesn't seem relevant. A few columns should be converted to integers. Because the income data carries information that disrupts the fairly consistent structure of the sales dataset, we will only grab the average household income for every town.
- [x] remove every second row
- [x] how many towns do we have
- [x] `population`, `list_year` and `total_cases` should be integers
- [x] drop all the rows with `NaN` towns if they exist
- [x] drop all rows after `2021-09-01`
- [x] sort table by months

In [178]:
# drop all rows after `2021-09-01`. (sales table only goes this far)
df2_tmp5 = df2_tmp4[df2_tmp4['month'] <= '2021-09-01']

# drop all remaining NA values
df2_tmp6 = df2_tmp5.dropna()

df2_tmp6[['list_year', 'population', 'total_cases']] = df2_tmp6[['list_year', 'population', 'total_cases']].astype(int)
df2 = df2_tmp6.sort_values(by='month')
display(df2.tail(20))

/tmp/ipykernel_50370/3954199287.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_tmp6[['list_year', 'population', 'total_cases']] = df2_tmp6[['list_year', 'population', 'total_cases']].astype(int)


,list_year,town,population,residential_type,assessed_value,sale_amount,month,total_cases,med_hsehld_income
27210,2020,New Canaan,20574,Single Family,705530.0,1290000.0,2021-08-31,27644,200203.0
27211,2020,New Canaan,20574,Single Family,536760.0,574972.0,2021-08-31,27644,200203.0
27212,2020,New Canaan,20574,Single Family,656530.0,1100000.0,2021-08-31,27644,200203.0
27213,2020,New Canaan,20574,Single Family,1305640.0,2150000.0,2021-08-31,27644,200203.0
27229,2020,New Canaan,20574,Single Family,728490.0,1275000.0,2021-08-31,27644,200203.0
27214,2020,New Canaan,20574,Condo,415310.0,667000.0,2021-08-31,27644,200203.0
27216,2020,New Canaan,20574,Single Family,2721530.0,4090000.0,2021-08-31,27644,200203.0
27217,2020,New Canaan,20574,Single Family,997780.0,1499000.0,2021-08-31,27644,200203.0
27218,2020,New Canaan,20574,Single Family,1585220.0,2898000.0,2021-08-31,27644,200203.0
27219,2020,New Canaan,20574,Single Family,570010.0,849000.0,2021-08-31,27644,200203.0


### Winding up and next steps

We have processed the data from all three datasets to produce a table that will help us analyze the effect of the coronavirus pandemic on Connecticut's residential real estate market, at least, as far as these data can allow us.

We are going to decide how to do this empirical examination, with what choice of tools and statistical models used in Economics. We are thinking of the hedonic model for now, and will probably proceed with it as it has been described to be popular with real estate analysis.

We conclude this section by saving this data in a .csv file.

In [179]:
df2.to_csv('./data/CTRRE_covid_income_2016-2021.csv')